# Exploration of Missed Tackles

## Why Missed Tackles

Tackling is a critical part of every football play. The one goal of the defense is to reach the ball carrier and bring them to the ground. This goal has two parts. First the defender must arrive in the same location as the ball carrier, but once they are there it is still imperitive to complete the tackle. Sometimes the defender is unable to bring the ball carrier to the ground and this is considered a missed tackle. This exploration considers the parts of the tackle and looks for indicators of missed tackles prior to contact.

## Metrics

### Parts of a Tackle

Tackles can be broken down to investigate different parts. First, is the need to reach the same point. Projecting the players direction to calculate the contact point can provide information about the possible contact. Such as the contact angle, the distance to contact, and the momentum of each player at contact. The momentum of each player can be further analyzed by determining the momentum in the x and y directions. Since the offense is attempting to advance the ball the ball carrier's momentum in the positive x direction will provide the most value and the defenders momentum in the negative x direction will be attempting to counter the positive x momentum of the ball carrier.

### Calculation of Metrics

All metrics were calculated based on a single frame from the tracking data. The calculations assume the frame represents a constant state that will continue until the defender and ball carrier make contact.

**Distance Between Players**

The euclidean distance between the ball carrier and every defensive player was calculated for each frame. The first frame where a defender came within 1 yard of the ball carrier was used to calculated the following metrics.

**Contact Point**

The direction of each player was used to determine the slope of the line that represents the players trajectory. With the slope of the trajectory and the position of the player, the equation of the line was calculated. After determining the equations of the trajectory lines for both the ball carrier and the defender, it is possible to solve for the intersection of these two lines which becomes the theoretical contact point.

**Contact is Behind the Players**

The theoretical point of contact is not limited to only the range of the field that is in front of both players because the trajectory line is not limitted to only extend in the direction the players are moving. Therefore, two boolean metrics were calculated to express whether the contact point was behind the ball carrier or behind the defender.

**Distance and Time to Contact**

The euclidean distance between the ball carrier and the contact point, as well as the distance between the defender and the contact point were calculated. To determine the time to reaching the contact point, the distance was divided by the current speed of the player.

**Momentum**

For each player the momentum was calculated by multiplying the player's weight by their speed. Momentum is conserved in collisions, so the simplified idea is that ball carriers with more momentum than tacklers could break through the tackle. Therefore, the difference in the ball carrier's momentum and the defender's momentum was calculated. To take this one step further, the momentum was broken down into it's x and y components. This specifies how much of the players momentum is moving toward the endzones (x component) and how much is moving toward the sidelines (y component). 

## Distinguishing Missed Tackles

Each defender was assigned a tag for every play. Either tackle (which included assists), missed tackle, or none. Forced fumbles were dropped from this evaluation. The metrics listed above were compared between these three groups to identify interesting trends. 

Speed of the tackler was higher (on average) for missed tackles than tackles.